In [ ]:
import itertools
import json
import numpy as np
import Hypergraph

lim_hyp_shuff=3
num_hyper_reshuff=1000

dataset='congress-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-committees_simplices';name_file='{0}'.format(dataset);
#dataset='house-committees_simplices';name_file='{0}'.format(dataset);
#dataset='email-Enron_simplices'; name_file='{0}'.format(dataset); 
#dataset='email-Eu_simplices'; name_file='{0}'.format(dataset); 
#dataset='hyperedges-cat-edge-algebra-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-geometry-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-music-blues-reviews'; name_file='{0}'.format(dataset);
#dataset='Mid1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Elem1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='InVS15'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='SFHH'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LH10'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LyonSchool'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Thiers13'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='M_PL_015_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_062_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);

with open('Data/{0}.json'.format(name_file)) as json_file:
    data = json.load(json_file)
    
data = [data[i] for i in range(len(data)) if len(data[i])>1] #select only interactions of size>=2
for i in range(len(data)):
    data[i].sort() #sort nodes in each interactions by their IDs, e.g. [2,1]->[1,2]
data.sort() 
data = list(data for data,_ in itertools.groupby(data)) #remove duplicated interactions
data.sort(key = len) 

L=len(data) 
size_max=len(data[-1])

M=np.zeros(size_max-1)

for j in range(len(data)): #list with number of hyperedges of size m in position m-2
    l = len(data[j])
    M[l-2]=M[l-2]+1

for run_id in range(num_hyper_reshuff):
    s_interactions=[]

    for k in range(len(M)):
        m=k+2
        hyperedgeList = [tuple(data[i]) for i in range(len(data)) if len(data[i])==m] #select interactions of size=m

        if M[k]<=lim_hyp_shuff: #do not perform the hyper-reshuffling if the number of hyperedges is too low (ineffective)
            ShufflingIterations=0
        else:
            ShufflingIterations = 10**5

        hypergraph = Hypergraph.HypergraphGenerator(hyperedgeList, type="hyperedge-list")
        hypergraph.shuffleHyperedges(m, ShufflingIterations)
        hyperedgeList = hypergraph.getHyperedgeList()

        for inter in hyperedgeList:
            s_interactions.append(inter)

    r_interactions=[list(s_interactions[i]) for i in range(len(s_interactions))]

    with open('Shuffled_{1}_{2}.json'.format(dataset,name_file,run_id), 'w') as f:
        json.dump(r_interactions, f)